<a target="_blank" href="https://colab.research.google.com/github/kircherlab/MPRAlib/examples/mpralib.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!pip --quiet install MPRAlib

# Loading Data and Understanding the MPRAlib data structure

Here we will load a barcode count file (output of [MPRAsnakeflow](https://github.com/kircherlab/MPRAsnakeflow)) into MPRAlib, more precise into an `MPRAData` data object in `mpralib.mpradata`. MPRAlib will generate an [AnnData](https://anndata.readthedocs.io) object out of it, which is acessable via `MRAData.data`:

<img src="https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg" alt="AnnData" width="480 "/>

## Core data

- `var`: barcodes
- `obs`: replicates

Oligo names are also stored in `var` like `MRAData.data.var["ologos"]`. Further DNA and RNA counts are stored in `layers` like `MRAData.data.layers[""dna"]` and `layers["rna"]`. This are the raw counst and those layers are never been modified.

Metadata is stored in the `MPRA.data.uns` dictionary.

## Extended count data

There are some further layers that are dynamically updated via the library. This are:

1. Normalization layers in `MRAData.data.layers["dna_normalized"]` and `MRAData.data.layers["rna_normalized"]`. Usually we have to normalize the data to account for different sequencing depths. This is done by dividing the raw counts by the sum of the counts all counts per replicate and scale them (usually `1e6` like counts per million).
2. Sampling layers in `MRAData.data.layers["dna_sampling"]` and `MRAData.data.layers["rna_sampling"]`. This is only an edge case when you want to downsample your counts.

The library usually generates the layers annd takes care of them. E.g. when barcode filtering is applied, the normalization layers are updated accordingly.

MPRAlib provides some getters (in python properties) to access the data intuitively. E.g. `MRAData.raw_dna_counts` will return the raw DNA counts. `MRAData.raw_rna_counts` for RNA accordingly. If you use the property `MPRAdata.rna_counts` or `MPRAdata.dna_counts` it will return the latest counts. That can be the raw counts (if nothing was done), Filtered counts when where some barcode counts are set to zero according to the barcode filter mask (see below), sampled or sampled and filtered counts. If you want to get norlazied counts use `MPRAdata.normalized_rna_counts` or `MPRAdata.normalized_dna_counts`. It returns the layer `"rna_normalized"` or `"dna_normalized"` which is a normalization of `MPRAdata.rna_counts` or `MPRAdata.dna_counts`.

## Barcode filtering

Barcode filters ar stored in a n_barcodes x n_replicates matrix within the AnnData object `MPRAData.data.varm["barcode_filter"]`. You can set any new filter using the setter `MPRAdata.barcode_filter = new_filter`. The filter is a boolean matrix with `True` for barcodes that should be removed and `False` for barcodes that should be kept. This setter magically also updates the normalized counts for you.

There are pre implemented methods to filter barcodes, like detecting outliers. We willcover thi part later.

## Grouping Data by Oligo

Usually when we work with MPRA data we are not interested by the barcode count itself but an aggregated count version per oligo. This is done by grouping the data by oligo and writing a new AnnData object into our library, called `MPRAdata.grouped_data`. It is a new AnnData object because the data structure is slightly different. The `var` is now the oligos and the `obs` are the replicates. The layers `"dna"`, `"rna"`, `"dna_normalized"`, and  `"rna_normalized"` exist also for the grouped data but are now aggregated per oligo. Also usually the filtered counts or sampled counts are used for the grouped data (if applicable). Also it uses a barcode threshold to filter out oligos that have not enough barcodes. This threshold can be set by `MPRAdata.barcode_threshold = 10` for example. The default is 1 (no Oligo is removed).

We have two additional layers: 1. `MPRAData.grouped_data.layers["log2FoldChange"]` stores the log2 rna/dna ratio. 2. `MPRAData.grouped_data.layers["barcodes"]` counts the number of barcodes per oligo that was used for teh aggregation.

## Example

Let's start with loading a barcode count file from [MPRAsnakeflow](https://github.com/kircherlab/MPRAsnakeflow):

In [2]:
# Loading the MPRAlib library
from mpralib.mpradata import MPRAdata
# Loading other libraries
import pandas as pd
import numpy as np

# Load the data
mpradata = MPRAdata.from_file("../resources/reporter_experiment.barcode.HEPG2.fromFile.default.all.tsv.gz")

# Getting counts, no filtering/sampling done, so raw counts
print("DNA counts")
display(pd.DataFrame(mpradata.dna_counts[:,0:5], index=mpradata.replicates, columns=mpradata.barcodes[0:5]))
print("RNA counts")
display(pd.DataFrame(mpradata.rna_counts[:,0:5], index=mpradata.replicates, columns=mpradata.barcodes[0:5]))


DNA counts


barcode,TACTCTCCGTGCCCA,GGTATAACATCTCCG,TTAGGAGTCACACGT,GAATATAACACCCGA,AAACACCGCGCTCTA
1,1,0,0,1,0
2,0,3,1,1,0
3,0,1,1,1,1


RNA counts


barcode,TACTCTCCGTGCCCA,GGTATAACATCTCCG,TTAGGAGTCACACGT,GAATATAACACCCGA,AAACACCGCGCTCTA
1,1,0,0,1,0
2,0,1,1,1,0
3,0,2,2,3,1


Generating correlations across repliactes on certain barcode thresholds:

In [3]:
print("Pairwise Pearson correlation")
display(pd.DataFrame(mpradata.pearson_correlation, index=mpradata.replicates, columns=mpradata.replicates).round(3))
print("Parwise Spearman correlation")
display(pd.DataFrame(mpradata.spearman_correlation, index=mpradata.replicates, columns=mpradata.replicates).round(3))


print("Mean Pearson correlation")
print(mpradata.pearson_correlation.flatten()[[1,2,5]].mean().round(3))
print("Mean Spearman correlation")
print(mpradata.spearman_correlation.flatten()[[1,2,5]].mean().round(3))


# Setting a different barcode threshold 10, 100
for threshold in [1, 10, 100]:
    mpradata.barcode_threshold = 10
    print(f"Mean Pearson correlation, BC threshold {mpradata.barcode_threshold}")
    print(mpradata.pearson_correlation.flatten()[[1,2,5]].mean().round(3))
    print(f"Mean Spearman correlation, BC threshold {mpradata.barcode_threshold}")
    print(mpradata.spearman_correlation.flatten()[[1,2,5]].mean().round(3))

Pairwise Pearson correlation


/home/mschubach/miniforge3/envs/jupyter/lib/python3.12/site-packages/mpralib/mpradata.py:433: RuntimeWarning: invalid value encountered in divide
  self.grouped_data.layers["rna_normalized"] / self.grouped_data.layers["dna_normalized"]


,1,2,3
1,1.000,0.456,0.469
2,0.456,1.000,0.481
3,0.469,0.481,1.000


Parwise Spearman correlation


,1,2,3
1,1.000,0.361,0.379
2,0.361,1.000,0.356
3,0.379,0.356,1.000


Mean Pearson correlation
0.469
Mean Spearman correlation
0.365
Mean Pearson correlation, BC threshold 10
0.655
Mean Spearman correlation, BC threshold 10
0.525
Mean Pearson correlation, BC threshold 10
0.655
Mean Spearman correlation, BC threshold 10
0.525
Mean Pearson correlation, BC threshold 10
0.655
Mean Spearman correlation, BC threshold 10
0.525


Using such strict barcode threshold will also reduce our oligos available per replicate:

In [ ]:
for threshold in [1, 10, 100]:
    mpradata.barcode_threshold = threshold
    print(f"Number of oligos per individual replicate, using BC threshold {mpradata.barcode_threshold}")
    print(np.sum(mpradata.grouped_data.layers["barcodes"]!=0,axis=1))


Number of oligos per individual replicate, using BC threshold 1


array([2243, 2250, 2253])

Number of oligos per individual replicate, using BC threshold 10


array([1359, 1349, 1357])

Number of oligos per individual replicate, using BC threshold 100


array([10,  3,  3])

## Barcode filtering and outlier detection

There is some pre-build function to remove barcodes from experiments. This is done via the function `MPRAData.apply_barcode_filter()`. This function has pre-build filters like setting a minimum or maximum threshold for counts. E.g. you want to remove very very high barcode counts or you 'trust' only barcode counts with 3 RNA counts to remove noisy data. The functin can also randomly remove barcodes if you want to downsample your data on barcodes (Which correspons to removing barcodes from your assignment, theerfor ehaving a lower quality/lower depth assignment file).

But it can also detect outlier barcodes. E.g. if you want to remove barcodes that do not follow the distribution per oligo. This is usually done per repliacte.

Now we try to remove barcodes that are outliers within the RNA counts using the zscore:

In [28]:
from mpralib.mpradata import BarcodeFilter

# using standard number of barcodes per oligo
mpradata.barcode_threshold = 10

# Resetting the barcode filter
mpradata.barcode_filter = None
print(f"Mean Pearson correlation before BC filter, BC threshold {mpradata.barcode_threshold}")
print(mpradata.pearson_correlation.flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpradata.grouped_data.layers["barcodes"]!=0,axis=1))

# Apply filter
mpradata.apply_barcode_filter(BarcodeFilter.RNA_ZSCORE, {"times_zscore": 3})

print(f"Mean Pearson correlation after BC filter, BC threshold {mpradata.barcode_threshold}")
print(mpradata.pearson_correlation.flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpradata.grouped_data.layers["barcodes"]!=0,axis=1))

Mean Pearson correlation before BC filter, BC threshold 10
0.655
Number of oligs
[1359 1349 1357]
Mean Pearson correlation after BC filter, BC threshold 10
0.594
Number of oligs
[1359 1349 1357]


You see we keeping number of oligos the same but our Person correlaton drops. Maybe within this example dataset this was not a good idea. Let's see if we achieve something else when allowing only a higher number of DNA and RNA counts:

In [40]:
mpradata.barcode_filter = None
print(f"Mean Pearson correlation before BC filter, BC threshold {mpradata.barcode_threshold}")
print(mpradata.pearson_correlation.flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpradata.grouped_data.layers["barcodes"]!=0,axis=1))

# Apply filter
mpradata.apply_barcode_filter(BarcodeFilter.MIN_COUNT, {"dna_min_count": 2, "rna_min_count": 4})

print(f"Mean Pearson correlation after BC filter, BC threshold {mpradata.barcode_threshold}")
print(mpradata.pearson_correlation.flatten()[[1,2,5]].mean().round(3))
print(f"Number of oligs")
print(np.sum(mpradata.grouped_data.layers["barcodes"]!=0,axis=1))

Mean Pearson correlation before BC filter, BC threshold 10
0.655
Number of oligs
[1359 1349 1357]
Mean Pearson correlation after BC filter, BC threshold 10
0.78
Number of oligs
[12  5  8]


Now we see an improved correlaton. But hold on... We have nearly no oligo left that fullfills the BC threshold 10 criteria. This example data seems to be a very very low count data. Maybe not the best idea. 